# Modelo de regressão (Forest Random) para a previsão da data de proxima compra

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
 
df = pd.read_csv('dataset_renomeado')

# 1. Converter colunas de data e hora
df['data_compra(a/m/d)'] = pd.to_datetime(df['data_compra(a/m/d)'])
df['hora'] = pd.to_datetime(df['hora_compra(h/m/s)'], format='%H:%M:%S', errors='coerce').dt.hour

# 2. Ordenar por cliente e data
df = df.sort_values(by=['id_cliente', 'data_compra(a/m/d)'])

# 3. Criar coluna com dias até a próxima compra
df['dias_ate_proxima_compra'] = df.groupby('id_cliente')['data_compra(a/m/d)'].shift(-1) - df['data_compra(a/m/d)']
df['dias_ate_proxima_compra'] = df['dias_ate_proxima_compra'].dt.days

# 4. Remover linhas sem próxima compra
df = df.dropna(subset=['dias_ate_proxima_compra'])

# 5. Features temporais
df['dia_semana'] = df['data_compra(a/m/d)'].dt.weekday
df['mes'] = df['data_compra(a/m/d)'].dt.month

# 6. Reduzir cardinalidade das colunas categóricas
high_card_cols = [
    'id_local_partida_ida',
    'id_local_destino_ida',
    'id_local_partida_volta',
    'id_local_destino_volta'
]

for col in high_card_cols:
    top_values = df[col].value_counts().nlargest(50).index
    df[col] = df[col].where(df[col].isin(top_values), 'outros')

# 7. Selecionar colunas para treino
colunas_usadas = [
    'id_local_partida_ida',
    'id_local_destino_ida',
    'id_local_partida_volta',
    'id_local_destino_volta',
    'empresa_onibus_ida',
    'empresa_onibus_volta',
    'valor_ticket',
    'qtd_tickets_compra',
    'dia_semana',
    'mes',
    'hora'
]

X = df[colunas_usadas]
y = df['dias_ate_proxima_compra']
 
# 8. Transformar variáveis categóricas em dummies
X = pd.get_dummies(X)
 
# 9. Dividir treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
 
# 10. Treinar modelo
model_reg = RandomForestRegressor()
model_reg.fit(X_train, y_train)
 
# 11. Avaliar desempenho
y_pred = model_reg.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f"Erro médio (MAE): {mae:.2f} dias")
 
# Agora você pode usar "model_reg" para prever a próxima compra